# Machens-Brody 2D discrimination model

We implement and simulate from the 2005 Machens-Romo-Brody 2 population spiking model (with the nullclines simulated via their mean field model). The spiking electrical activity is then simulated to be observed through a fictional non-saturating, fixed onset and decay time calcium reporter with parameters ...

We are using the provided Matlab code to simulate the spiking dynamics, with slight changes to the simulation:
- We simulate the mean field dynamics as is (Euler model with dt=1msec), but injecting additive noise at the beginning (par.init_Sigma) and also (par.noise_eps) every par.noiseTimeStep msec, as is in our model
- The loading dataset uses the dynamics only when f1 is active, and the decision dataset is simulated when only f2 is active [both starting from (0,0) initial conditions]
- Only ambigous trials, type [4,4] in the code

See the parameter initialisations in ```thesis_initpar_loading.m``` and ```thesis_initpar_decision.m```, and the simulation process in ```spikesim_special.m``` and ```mfsim_special.m```. The datasets were created by ```thesis_create_ExperimentData_spiking.m```.



In [ ]:
import numpy as np

import scipy.linalg
import scipy.cluster
import scipy.stats
from collections import OrderedDict
import random

import plotly
from plotly.offline import iplot as plt
from plotly import graph_objs as plt_type
plotly.offline.init_notebook_mode(connected=True)

from IPython.core.debugger import set_trace



In [ ]:
import nbimporter

# # Import 1d well examples
from GPDM_direct_fixedpoints import *

# plotly.offline.init_notebook_mode(connected=True)

# from joblib import Parallel, delayed
# import multiprocessing
# import itertools

# num_cores = multiprocessing.cpu_count()

In [ ]:
import scipy.optimize
import pickle
import pickle, datetime, time

In [ ]:
import scipy.io

In [ ]:
data_loading_uneven = scipy.io.loadmat('thesis_Machens_Brody_sim/thesis_MachensBrodySim_20181230T121236_loading_uneven.mat', squeeze_me=True)
data_loading_even = scipy.io.loadmat('thesis_Machens_Brody_sim/thesis_MachensBrodySim_20181230T121236_loading_even.mat', squeeze_me=True)
data_decision = scipy.io.loadmat('thesis_Machens_Brody_sim/thesis_MachensBrodySim_20181230T121236_decision.mat', squeeze_me=True)

# data_loading_uneven = scipy.io.loadmat('thesis_Machens_Brody_sim/thesis_MachensBrodySim_20181129T172557_loading_uneven.mat', squeeze_me=True)
# data_loading_even = scipy.io.loadmat('thesis_Machens_Brody_sim/thesis_MachensBrodySim_20181129T172557_loading_even.mat', squeeze_me=True)
# data_decision = scipy.io.loadmat('thesis_Machens_Brody_sim/thesis_MachensBrodySim_20181129T172557_decision.mat', squeeze_me=True)

# Simulate calcium traces from the spike times

In [ ]:
def fred_curve(fred_params):
    # Fast rise exponential decay model, A, tau1, tau2
    t = 1+np.arange(999) # msec
    out = ( 
        np.sqrt(np.exp(2*fred_params['tau1']/fred_params['tau2'])) * 
        np.exp(-fred_params['tau1']/t - t/fred_params['tau2'])
    )
    
    t = np.concatenate([np.array([0]), t])
    out = np.concatenate([np.array([0]), out])
    
    out = (fred_params['A']) * (out / np.max(out))
     
    return t, out

In [ ]:
# Set params, load and resample the data
from collections import OrderedDict


simParams = OrderedDict()
simParams['seed'] = 1234
simParams['curData'] = 'decision'  # decision or loading_even or loading_uneven 
if simParams['curData'] == "loading_even":
    curData = data_loading_even
elif simParams['curData'] == "loading_uneven":
    curData = data_loading_uneven
else:
    curData = data_decision

simParams['trial_nums'] = range(40)
simParams['delta_t'] = 125 # msec sampling (8 Hz)
simParams['T_max'] = 2001
simParams['Sigma_calc_obs'] = 1e-2
simParams['baseline_brightness'] = 1.
simParams['fred_params'] = OrderedDict(A=0.19, tau1=45./2, tau2=142.) #OrderedDict(A=1, tau1=1, tau2=3.)#OrderedDict(A=1, tau1=6.5, tau2=120.)




def createData(curData, simParams, batchnum=0):
    
    # Setup the initial observations in y
    obs_times = curData['data']['rt'][0][range(0, min(simParams['T_max'], curData['data']['rt'][0].shape[0]), simParams['delta_t'])]
    
    #y_spikes = []
    
    y_calcium = (
        np.ones(((curData['par']['Nneurons']).astype(int)*2, len(obs_times), len(simParams['trial_nums'])))
        * simParams['baseline_brightness']
    )
    
    # Get the convultion kernel
    conv_t, conv_weights = fred_curve(simParams['fred_params'])

    # Gather data from each trial
    for n_trial in simParams['trial_nums']:
        
        
        
        # Go from spike times on single neurons to calcium dF/F type signals, sampled at low frequency
        flourescence_cur_trial = np.zeros(((curData['par']['Nneurons']).astype(int)*2, len(curData['data']['rt'][0])))

        # For each spike on each neuron, add the conv_weights appropriately
        cur_spikes = np.concatenate([curData['data']['spikesx'][n_trial], curData['data']['spikesy'][n_trial]])
        
        for m_neuron in range(cur_spikes.shape[0]):
            for s_spike in range(cur_spikes.shape[1]):
                if cur_spikes[m_neuron,s_spike]==0: # No more spike times on that neuron
                    break

                time_ind = int(np.floor(cur_spikes[m_neuron,s_spike]))
                flourescence_cur_trial[m_neuron, time_ind:min(time_ind+len(conv_weights),flourescence_cur_trial.shape[1])] += (
                    conv_weights[0:(min(len(conv_weights),flourescence_cur_trial.shape[1]-time_ind))]
                )
            
        # Questions: Mutliplicative? Point observation or time-gated sum or total cumulative sum?
        # I think multiplicative point observations with added "gaussian" noise subsequently
        y_calcium[:,:,n_trial] *= (1+flourescence_cur_trial[:, obs_times])
        
    # Add observation noise to y
    y_calcium = y_calcium + np.sqrt(simParams['Sigma_calc_obs'])*np.random.randn(*y_calcium.shape)
    
    return y_calcium #, y_spikes # Add spikes output later (for Lea's code)
    

### Create various visualisations to describe the simulation process



In [ ]:
import copy

In [ ]:
# Default convolution curve
visParams = copy.deepcopy(simParams)
tmp = fred_curve(visParams['fred_params'])
plt([plt_type.Scatter(x=tmp[0], y=tmp[1])])

In [ ]:
# Create "binned spike time" data
visParams = copy.deepcopy(simParams)
visParams['fred_params'] = OrderedDict(A=1, tau1=0.1, tau2=0.1)
tmp = fred_curve(visParams['fred_params'])
plt([plt_type.Scatter(x=tmp[0], y=tmp[1])])

visParams['delta_t'] = 1
visParams['Sigma_calc_obs'] = 0
y = createData(curData, visParams, batchnum=0) - 1

tmp_time_bin = 125 # ms
tmp_obs_times = curData['data']['rt'][0][range(0, min(visParams['T_max'], curData['data']['rt'][0].shape[0]), tmp_time_bin)]
    
y = np.cumsum(y, axis=1)

y = np.diff(y[:, tmp_obs_times, :], axis=1)

# An example + and - neuron vs time on one trial 
all_plots = []

for n_trial in [1]:
    all_plots.append(plt_type.Scatter(x=np.arange(y.shape[1]), y = y[:1, :, n_trial].mean(0)))
    all_plots.append(plt_type.Scatter(x=np.arange(y.shape[1]), y = y[-1:, :, n_trial].mean(0)))
    #all_plots.append(plt_type.Scatter(x=y[:30,:, n_trial].mean(0), y=y[-30:,:, n_trial].mean(0)))
    
plt(all_plots)


# An example + and - neuron vs time on one trial 
all_plots = []

# Show population average for the two types of neurons, plus the individual traces

for n_trial in [1]:
    for n_neuron in range(30):
        all_plots.append(plt_type.Scatter(x=np.arange(y.shape[1]), y = y[n_neuron, :, n_trial], mode='lines', line=dict(color='gray', width=0.5)))
    for n_neuron in range(30,60):
        all_plots.append(plt_type.Scatter(x=np.arange(y.shape[1]), y = y[n_neuron, :, n_trial], mode='lines', line=dict(color='gray', width=0.5)))
    
    all_plots.append(plt_type.Scatter(x=np.arange(y.shape[1]), y = y[range(30), :, n_trial].mean(0), mode='lines', line=dict(color='blue')))
    all_plots.append(plt_type.Scatter(x=np.arange(y.shape[1]), y = y[range(30,60), :, n_trial].mean(0), mode='lines', line=dict(color='red')))

    #all_plots.append(plt_type.Scatter(x=np.arange(y.shape[1]), y = y[-1:, :, n_trial].mean(0)))
    #all_plots.append(plt_type.Scatter(x=y[:30,:, n_trial].mean(0), y=y[-30:,:, n_trial].mean(0)))
    
plt(all_plots)

In [ ]:
# Create data with no noise
visParams = copy.deepcopy(simParams)
visParams['delta_t'] = 1
visParams['Sigma_calc_obs'] = 0
y = createData(curData, visParams, batchnum=0)

# An example + and - neuron vs time on one trial 
all_plots = []

for n_trial in [1]:
    all_plots.append(plt_type.Scatter(x=np.arange(y.shape[1]), y = y[:1, :, n_trial].mean(0)))
    all_plots.append(plt_type.Scatter(x=np.arange(y.shape[1]), y = y[-1:, :, n_trial].mean(0)))
    #all_plots.append(plt_type.Scatter(x=y[:30,:, n_trial].mean(0), y=y[-30:,:, n_trial].mean(0)))
    
plt(all_plots)

In [ ]:
# Show typical noise level at 1 ms bins
visParams = copy.deepcopy(simParams)
y = createData(curData, visParams, batchnum=0)

# An example + and - neuron vs time on one trial 
all_plots = []

# Show population average for the two types of neurons, plus the individual traces

for n_trial in [1]:
    for n_neuron in range(30):
        all_plots.append(plt_type.Scatter(x=np.arange(y.shape[1]), y = y[n_neuron, :, n_trial], mode='lines', line=dict(color='gray', width=0.5)))
    for n_neuron in range(30,60):
        all_plots.append(plt_type.Scatter(x=np.arange(y.shape[1]), y = y[n_neuron, :, n_trial], mode='lines', line=dict(color='gray', width=0.5)))
    
    all_plots.append(plt_type.Scatter(x=np.arange(y.shape[1]), y = y[range(30), :, n_trial].mean(0), mode='lines', line=dict(color='blue')))
    all_plots.append(plt_type.Scatter(x=np.arange(y.shape[1]), y = y[range(30,60), :, n_trial].mean(0), mode='lines', line=dict(color='red')))

    #all_plots.append(plt_type.Scatter(x=np.arange(y.shape[1]), y = y[-1:, :, n_trial].mean(0)))
    #all_plots.append(plt_type.Scatter(x=y[:30,:, n_trial].mean(0), y=y[-30:,:, n_trial].mean(0)))
    
plt(all_plots)

### Create the dataset used for fitting

In [ ]:
y = createData(curData, simParams, batchnum=0)

In [ ]:
# An example + vs - neuron on each trial
all_plots = []
for n_trial in range(y.shape[2]):
    all_plots.append(plt_type.Scatter(x=y[:1,:, n_trial].mean(0), y=y[-1:,:, n_trial].mean(0)))
    #all_plots.append(plt_type.Scatter(x=y[:30,:, n_trial].mean(0), y=y[-30:,:, n_trial].mean(0)))
    
plt(all_plots)

In [ ]:
# Plot the nullclines and the data
plt(plt_type.Figure(data=
    [plt_type.Scatter(x=curData['nullcline1'][0,:], y=curData['nullcline1'][1,:]),
     plt_type.Scatter(x=curData['nullcline2'][0,:], y=curData['nullcline2'][1,:])
    ], 
                    layout=plt_type.Layout(xaxis={'range': [0.,8.]}, yaxis={'range': [0.,8.]})))

plots_by_run = []
for v in range(y.shape[2]):
    plots_by_run.append(
        plt_type.Scatter(x=np.squeeze(y[0,:,v]), 
                      y=np.squeeze(y[-1,:,v]), 
                      mode='lines')
    )
    
plt(plots_by_run)

In [ ]:
# Define a plotting function for callback that shows the current transition function estimate
import plotly.figure_factory

def machens_callback_plot_external(pvec_partial, 
                                  opt_params, init_paramvec, transforms, dict_ind, dict_shape, nullclines=None
                                 ):
    
    paramvec = replace_params(pvec_partial, opt_params, init_paramvec)
    paramdict = vec_to_params(paramvec, dict_ind, dict_shape, transforms)
       
    # Unpack the usual parameters
    (Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J)  = \
        list(paramdict.values())[:12]
    
    if np.any(np.isnan(lengthscales)):
        set_trace()
    
    # Deal with the extra possible parameters
    Sigma_s = None; Sigma_J=None;
    if 'Sigma_s' in paramdict.keys():
        Sigma_s = paramdict['Sigma_s']
    if 'Sigma_J' in paramdict.keys():
        Sigma_J = paramdict['Sigma_J']

    # Plot transition function
    xtmp, ytmp = np.meshgrid(np.linspace(np.min(z[0,:]),np.max(z[0,:]),30),
                             np.linspace(np.min(z[1,:]),np.max(z[1,:]),30))
    xstar = np.concatenate([xtmp.flatten()[:,None], ytmp.flatten()[:,None]], axis=1).T

    L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=lengthscales, z=z, u=u, s=s, J=J, 
                                         sig_eps=Sigma_eps, sig_u=Sigma_u, sig_s=Sigma_s, sig_J = Sigma_J)
    mu_star, sig_star, K_pred = fp_predict(xstar, L, targets, params)

    #print(time_full_iter(pvec, y, D, Nz, Ns)[0])
    
    quiver_fig = plotly.figure_factory.create_quiver(np.squeeze(xstar[0,:]), 
                                                     np.squeeze(xstar[1,:]), 
                                                     np.squeeze(mu_star[0,:]-xstar[0,:]), 
                                                     np.squeeze(mu_star[1,:]-xstar[1,:]),
                                                    scale=.25,
                                                    arrow_scale=.4,)

#     quiver_fig = plotly.figure_factory.create_quiver(np.squeeze(y_t1_rsh[0,:]), 
#                                                  np.squeeze(y_t1_rsh[1,:]), 
#                                                  np.squeeze(y_t_rsh[0,:]-y_t1_rsh[0,:]), 
#                                                  np.squeeze(y_t_rsh[1,:]-y_t1_rsh[1,:]),
#                                                 scale=.25,
#                                                 arrow_scale=.4,)
    
    # Add points to figure
    
    # Map Sigma_s values to the range 12-22
    if Sigma_s.size>1:
        FP_SIZE = -np.squeeze(np.log(Sigma_s))
        FP_SIZE = FP_SIZE - np.min(FP_SIZE)
        FP_SIZE = (FP_SIZE/np.max(FP_SIZE))*10. + 12.
    else:
        FP_SIZE = 12.
    
    # Inducing point locations
    quiver_fig['data'].append(
        plt_type.Scatter(x=np.atleast_1d(np.squeeze(z[0,:])), y=np.atleast_1d(np.squeeze(z[1,:])), 
                         mode='markers', name="Inducing loc", marker=dict(size=10)))
    
    # Estimated fixed points
    quiver_fig['data'].append(
        plt_type.Scatter(x=np.atleast_1d(np.squeeze(s[0,:])), y=np.atleast_1d(np.squeeze(s[1,:])), 
                         mode='markers', name="Fixed loc", marker=dict(size=FP_SIZE)))
        
    # Add nullclines    
    if nullclines is not None:
        quiver_fig['data'].append(plt_type.Scatter(x=nullclines['nullcline1'][0,:], y=nullclines['nullcline1'][1,:]))
        quiver_fig['data'].append(plt_type.Scatter(x=nullclines['nullcline2'][0,:], y=nullclines['nullcline2'][1,:]))
        
    
    # Add layout
    quiver_fig['layout'] = plt_type.Layout(xaxis={'range': [-1.,8.]}, yaxis={'range': [-1.,8.]})
    
    
    plt(quiver_fig)

# Single experiment in decision making data

In [ ]:
y.shape

In [ ]:
y_train = y
y_means = np.stack([y[:30,:, :].mean(0), y[-30:,:, :].mean(0)])
x_train = []

In [ ]:
y_train.shape

In [ ]:
D = 2
Nz = 36
Ns = 5

#######################################################
# Initialise the parameters
paramdict = init_params(y_train, D, Nz, Ns, grad_sigma=True)


In [ ]:
paramdict

In [ ]:
# Add transformations for certain parameters 
# (Note that the parameter indices may be changed by transforms! (for cholesky repres of matrices, "SquareMatrix" type))
transforms = OrderedDict()
for par in ['Sigma_0_0', 'Sigma_u', 'Sigma_s', 'Sigma_J', 'lengthscales', 'Sigma_eps', 'Sigma_nu', 'kernel_variance']:
    transforms[par] = {}
    transforms[par]['type'] = "Square"

# Create vectorised and transformed representation
(init_paramvec, dict_ind, dict_shape) = params_to_vec(paramdict, transforms=transforms)

#######################################################
# Optimise only certain elements of paramvec (messy with indices)
opt_params = np.arange(init_paramvec.shape[0])
opt_params = np.delete(opt_params, np.hstack([dict_ind['C'], dict_ind['z']])) # All except the ones listed here
cur_pvec = init_paramvec[opt_params]

In [ ]:



#######################################################
# Add bounds for parameters 
bnds = list(((None, None),) * init_paramvec.shape[0])

bnds_final = []
for i in opt_params:
    bnds_final.append(bnds[i])
bnds = tuple(bnds_final)

#######################################################
# Add priors (to span at least the bounds)
priors = []

# Add prior to ensure inducing points are smooth
cur_prior = {}
cur_prior['type'] = "InducingSmooth_and_DPP"
cur_prior['metadata'] = {}
def unpack_dict_tmp(pdict):
    kernelparams = {'lengthscales': pdict['lengthscales'], 'kernel_variance': pdict['kernel_variance']}
    # Return the parameters we want in the required format (joint smoothness of (z-u) and s)
    return (np.concatenate([pdict['u'], pdict['s']],axis=1), 
            np.concatenate([pdict['z'], pdict['s']],axis=1),
            np.concatenate([pdict['Sigma_u'], pdict['Sigma_s']]),
            kernelparams)
cur_prior['metadata']['unpack_dict'] = unpack_dict_tmp
cur_prior['metadata']['kernel_func'] = RBF
cur_prior['metadata']['prior_weight_Smooth'] = 1e0
cur_prior['metadata']['prior_weight_DPP'] = 1e0
priors.append(cur_prior)


# Prepare the optimisation
f_objective = lambda pvec_partial: (time_full_iter(replace_params(pvec_partial, opt_params, init_paramvec), 
                                            y_train, dict_ind, dict_shape, 
                                            transforms=transforms,
                                            priors=priors)[0])
f_minibatch = lambda pvec_partial: (minibatch_iter(replace_params(pvec_partial, opt_params, init_paramvec), 
                                            y_train, dict_ind, dict_shape, 
                                            transforms=transforms,
                                            priors=priors,
                                            minibatch_size=3)[0])
f_minibatch_with_grad = grad(f_minibatch, argnum=0)

f_objective_with_grad = grad(f_objective, argnum=0)



# By iterating minimize within a for cycle, we can save all intermediate results and set ending times
start_time = datetime.datetime.now().strftime("%Y%m%dT%H%M%S")
save_fname_params = ""
save_fname = "Experiment_machens_results/machens_" + simParams['curData'] +"_" + start_time + save_fname_params + ".pkl"
init_time = time.time()
max_time = 2.0*3600 # Maximum iteration time in seconds, break if reached
all_results = []

saving_callback_params = [save_fname, y_train, x_train, simParams,
                     init_paramvec, dict_ind, dict_shape, opt_params, 
                     bnds, transforms]
def saving_callback_func(cur_result, save_fname, y_train, x_train, simParams,
                     init_paramvec, dict_ind, dict_shape, opt_params, 
                     bnds, transforms):
    # Saving every 10 iterations
    if ~np.mod(cur_result.iters[-1], 10):
        pickle.dump([y_train, x_train,
                     cur_result, simParams,
                     init_paramvec, dict_ind, dict_shape, opt_params, 
                     bnds, transforms], open(save_fname, 'wb'))
        


# for it in range(100):
#     result = scipy.optimize.minimize(f_objective_with_grad, cur_pvec, jac=True, method='L-BFGS-B', bounds=bnds, callback=None,
#                           options={'maxiter':10, 'disp':True})
for it in range(5):
    np.random.seed(1234+it)
    result = adamOptimize(f_objective, f_minibatch_with_grad, cur_pvec,
                      options={'obj_eval_iters': 5, 'maxiter':200, 'disp':True},
                         callback_func=saving_callback_func, callback_params=saving_callback_params)
    all_results.append(result)
    # Save the results
    with open(save_fname, 'wb') as f:
        pickle.dump([y_train, x_train,
                     all_results, simParams,
                     init_paramvec, dict_ind, dict_shape, opt_params, 
                     bnds, transforms], f)
    cur_pvec = result.x
    cur_time = time.time()

    machens_callback_plot_external(cur_pvec, 
                              opt_params, init_paramvec, transforms, dict_ind, dict_shape
                             )

    # Exit if maximum time is reached
    if ((cur_time - init_time) > max_time):
        print(["Maximum iteration time reached at iter", it])
        break

#         if len(all_results)>=2:
#             if (all_results[-1].fun - all_results[-2].fun) >= (-1e-2*num_trials):
#                 print(["Update did not improve objective function, stopping"])
#                 break


In [ ]:
%debug

In [ ]:
machens_callback_plot_external(cur_pvec, 
                              opt_params, init_paramvec, transforms, dict_ind, dict_shape
                             )

In [ ]:
os.write(1, "bla".encode())

In [ ]:
final_paramvec = replace_params(all_results[-1].x, opt_params, init_paramvec)
final_paramdict = vec_to_params(final_paramvec, dict_ind, dict_shape, transforms)

# Unpack the usual parameters
(Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J, tmp1212, tmp1234)  = \
    final_paramdict.values()

In [ ]:
final_paramdict

# Examine results

In [ ]:
cur_pvec = result.x
cur_time = time.time()

In [ ]:
#machens_callback_plot(init_paramvec[opt_params])

In [ ]:
#machens_callback_plot(cur_pvec)

In [ ]:
paramvec = init_paramvec
paramdict = vec_to_params(paramvec, dict_ind, dict_shape, transforms)

# Unpack the usual parameters
(Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J, Sigma_s, Sigma_J)  = \
    paramdict.values()
    
np.set_printoptions(precision=8)
print(np.concatenate([s.T, Sigma_s], axis=1))

paramdict

In [ ]:

paramvec = replace_params(all_results[-1].x, opt_params, init_paramvec)
paramdict = vec_to_params(paramvec, dict_ind, dict_shape, transforms)

# Unpack the usual parameters
(Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J, Sigma_s, Sigma_J)  = \
    paramdict.values()
    
np.set_printoptions(precision=8)
print(np.concatenate([s.T, Sigma_s], axis=1))

paramdict

In [ ]:
s.T

In [ ]:
Sigma_s

In [ ]:
np.linalg.eig(J[0,:,:])[1][:,1]

In [ ]:
Sigma_J

# Plotting individual trajectories and estimates

In [ ]:
%run "GPDM_direct_fixedpoints.ipynb"

In [ ]:
# Plot smoothed trajectories
obj, x_t1, x_t, sig_t1, sig_t, negll_all, _ = time_full_iter(replace_params(all_results[-1].x, opt_params, init_paramvec), 
                                               y, dict_ind, dict_shape, transforms=transforms, ret_smoothed=True)


In [ ]:
np.unravel_index(np.argmin(negll_all), negll_all.shape)

In [ ]:
plt([plt_type.Histogram(x=np.squeeze(negll_all.flatten()))])

In [ ]:

def plot_trajectories(y,x,
                      x_t1, x_t, sig_t1, sig_t,
                      v,
                      ax_pixel = 700,
                      ax_range = (-0.5,6.5),
                      user_scale = 1.0,
                      marker_op=0.3):

    
    # Use proper scaling of the markers to represent standard deviation (or a fraction of it based on user-scale)
    std_scale = user_scale * (ax_pixel/(ax_range[1]-ax_range[0]))
    
    plots_by_run = [] 
    plots_by_run.append(
        plt_type.Scatter(x=np.squeeze(y[0,:,v]), 
                      y=np.squeeze(y[1,:,v]), 
                      mode='lines',
                        name='y')
    )
    
#     plots_by_run.append(
#         plt_type.Scatter(x=np.squeeze(x[0,:,v]), 
#                       y=np.squeeze(x[1,:,v]), 
#                       mode='lines')
#     )

    plots_by_run.append(
        plt_type.Scatter(x=np.squeeze(x_t[0,:,v]), 
                      y=np.squeeze(x_t[1,:,v]), 
                      mode='lines',
                        name ='x_t')
    )
    
    plots_by_run.append(
        plt_type.Scatter(x=np.squeeze(x_t1[0,:,v]), 
                      y=np.squeeze(x_t1[1,:,v]), 
                      mode='markers',
                    marker=dict(size=np.squeeze(std_scale*np.sqrt(np.mean(sig_t1[[0,3],:,v], axis=0))),
                               opacity=marker_op),
                        name = 'x_t1')
    )
    
    
    layout1 = plt_type.Layout(
        width=ax_pixel, 
        height=ax_pixel,
        xaxis=dict(range=ax_range),
        yaxis=dict(range=ax_range)
    )
    
    plt(plt_type.Figure(data=plots_by_run, layout=layout1))


    layout2 = plt_type.Layout(
        height=ax_pixel,
        yaxis=dict(range=ax_range)
    )
    

    plt_data = [
        plt_type.Scatter(x=np.squeeze(np.arange(y.shape[1])), 
                      y=np.squeeze(y[0,:,v]), 
                      mode='lines', name='y_0'),
        
        plt_type.Scatter(x=np.squeeze(np.arange(y.shape[1])), 
                      y=np.squeeze(y[1,:,v]), 
                      mode='lines', name='y_1'),
    
                plt_type.Scatter(x=np.squeeze(np.arange(y.shape[1])), 
                      y=np.squeeze(x_t[0,:,v]), 
                      mode='lines', name='xt_0'),
                plt_type.Scatter(x=np.squeeze(np.arange(y.shape[1])), 
                      y=np.squeeze(x_t[1,:,v]), 
                      mode='lines', name='xt_1'),
            
            plt_type.Scatter(x=np.squeeze(np.arange(y.shape[1])), 
                      y=np.squeeze(x_t1[0,:,v]), 
                      mode='lines+markers', name='xt1_0',
                            marker=dict(size=np.squeeze(std_scale*np.sqrt(sig_t1[0,:,v])),
                                       opacity=marker_op)),
        
                        plt_type.Scatter(x=np.squeeze(np.arange(y.shape[1])), 
                      y=np.squeeze(x_t1[1,:,v]), 
                      mode='lines+markers', name='xt1_1',
                            marker=dict(size=np.squeeze(std_scale*np.sqrt(sig_t1[-1,:,v])),
                                        opacity=marker_op)
                                        )
    ]
    
    plt(plt_type.Figure(data=plt_data, layout=layout2))

In [ ]:
plot_trajectories(y_means,[],
                      x_t1, x_t, sig_t1, sig_t,
                      1)

In [ ]:
tcur = 5; v = 17

In [ ]:
Sigma_eps

In [ ]:
sig_t1[:,tcur:(tcur+1),v]

In [ ]:
sig_t1[:,tcur:(tcur+1),v]

In [ ]:
Sigma_t_t1 = np.reshape(sig_t1[:,tcur:(tcur+1),v],(2,2))

In [ ]:
mu_t_t1 = x_t1[:,tcur:(tcur+1),v]

In [ ]:
y_t = y[:,tcur:(tcur+1),v]

In [ ]:
mu_gauss = np.dot(C, mu_t_t1) 
var_gauss = np.dot(np.dot(C, Sigma_t_t1), C.T) + np.diag(Sigma_nu.flatten())

# Check for condition number, if high compared to expected precision in y (Sigma_nu), use pseudo-inverse and pseudo-determinant
prec = np.mean(np.log(1./Sigma_nu))
prec = prec-1
if np.log(np.linalg.cond(var_gauss)) > prec:
    inv_var_gauss = np.linalg.pinv(var_gauss, rcond=(1./prec))
    ldet = np.abs(np.linalg.eigvals(var_gauss))
    ldet = np.sum(np.log(ldet[np.max(ldet)>(1./prec)]))
else:    
    inv_var_gauss = np.linalg.inv(var_gauss)
    ldet = np.linalg.slogdet(var_gauss)[1]

D = mu_t_t1.shape[1]*1.0

log_marg_ll = (
    - 1.0* D/2.0*np.log(2*np.pi)
    - 1.0/2.0*ldet
    - 1.0/2.0*np.dot(np.dot( (y_t.T - mu_gauss.T), inv_var_gauss), (y_t - mu_gauss) )  
)

In [ ]:
np.abs(np.linalg.eigvals(var_gauss))/np.max(np.abs(np.linalg.eigvals(var_gauss)))

In [ ]:
prec = np.mean(np.log(1./Sigma_nu))

In [ ]:
np.log(np.linalg.cond(var_gauss))

In [ ]:
prec

In [ ]:
np.log(np.linalg.cond(var_gauss)) > prec

In [ ]:
log_marg_ll

In [ ]:
inv_var_gauss = np.linalg.inv(var_gauss)
ldet = np.linalg.slogdet(var_gauss)[1]

In [ ]:
np.linalg.slogdet(var_gauss)

In [ ]:
- 1.0/2.0*np.dot(np.dot( (y_t.T - mu_gauss.T), inv_var_gauss), (y_t - mu_gauss))

In [ ]:
ldet = np.linalg.svd(var_gauss, compute_uv=False)
ldet = np.sum(np.log(ldet[ldet>(1./prec)]))

In [ ]:
np.linalg.svd(var_gauss, compute_uv=False)

In [ ]:
ldet

In [ ]:
ldet = np.linalg.svd(var_gauss, compute_uv=False)
ldet = np.sum(np.log(ldet[ldet>(1./prec)]))
- 1.0/2.0*ldet

In [ ]:
inv_var_gaussa

In [ ]:
y_t - mu_gauss

In [ ]:
np.linalg.svd(var_gauss, compute_uv=False)

In [ ]:
ldet[ldet>(1./prec)]

In [ ]:
log_marg_ll

In [ ]:
nlog_marg_ll(mu_t_t1, Sigma_t_t1, C, Sigma_nu, y_t)

In [ ]:
np.diag(Sigma_nu.flatten())

In [ ]:
np.dot(np.dot(C, Sigma_t_t1), C.T)

In [ ]:
np.linalg.cond(Sigma_t_t1 + np.diag(Sigma_nu.flatten()))

In [ ]:
replace_params(all_results[-1].x, opt_params, init_paramvec)[dict_ind['Sigma_eps']]

In [ ]:
prec = np.mean(np.log(1./Sigma_nu))

In [ ]:
prec = np.mean(np.log(1./Sigma_nu))
np.log(np.linalg.cond(var_gauss))

In [ ]:
inv_var_gauss = np.linalg.inv(var_gauss)
ldet = np.linalg.slogdet(var_gauss)[1]
print(inv_var_gauss)
print(ldet)

In [ ]:
np.linalg.svd(var_gauss, compute_uv=False)**2

In [ ]:
np.linalg.eigvals(var_gauss)

In [ ]:
np.linalg.cond(var_gauss)

In [ ]:
1./prec

In [ ]:
inv_var_gauss = np.linalg.pinv(var_gauss, rcond=(1./prec))
ldet = np.abs(np.linalg.eigvals(var_gauss))
ldet = np.sum(np.log(ldet[(ldet/np.max(ldet))>(1./prec)]))
print(inv_var_gauss)
print(ldet)

In [ ]:
D = 2.0

In [ ]:
log_marg_ll = (
    - 1.0* D/2.0*np.log(2*np.pi)
    - 1.0/2.0*np.linalg.slogdet(var_gauss)[1] 
    - 1.0/2.0*np.dot(np.dot( (y_t.T - mu_gauss.T), inv_var_gauss), (y_t - mu_gauss) )  
)
print log_marg_ll

In [ ]:
mu_gauss = np.dot(C, mu_t_t1) 
var_gauss = np.dot(np.dot(C, Sigma_t_t1), C.T) + np.diag(Sigma_nu.flatten())

inv_var_gauss = np.linalg.inv(var_gauss)

D = mu_t_t1.shape[1]*1.0

log_marg_ll = (
    - 1.0* D/2.0*np.log(2*np.pi)
    - 1.0/2.0*np.linalg.slogdet(var_gauss)[1] 
    - 1.0/2.0*np.dot(np.dot( (y_t.T - mu_gauss.T), inv_var_gauss), (y_t - mu_gauss) )  
)

In [ ]:
var_gauss = np.dot(np.dot(C, Sigma_t_t1), C.T) + np.diag(Sigma_nu.flatten())

In [ ]:
np.linalg.cond(var_gauss)

In [ ]:
np.exp(np.linalg.slogdet(var_gauss)[1])

In [ ]:
Sigma_t_t1

In [ ]:
(y_t.T - mu_gauss.T)

In [ ]:
np.linalg.slogdet(var_gauss)

In [ ]:
proj_var = np.dot(C, Sigma_t_t1)
proj_inv = np.linalg.inv(np.dot(proj_var, C.T) + np.diag(Sigma_nu.flatten()))
proj_back = np.dot(Sigma_t_t1, np.dot(C.T, proj_inv))

mu_t_t = mu_t_t1 + np.dot(proj_back, (y_t - np.dot(C, mu_t_t1)))

Sigma_t_t = Sigma_t_t1 - np.dot(proj_back, proj_var)

In [ ]:
y_t - np.dot(C, mu_t_t1)

In [ ]:
proj_back

In [ ]:
np.diag(Sigma_nu.flatten())

In [ ]:
np.linalg.cond(np.dot(proj_var, C.T)+ np.diag(Sigma_nu.flatten()))

In [ ]:
proj_inv

In [ ]:
y_t

In [ ]:
mu_t_t1

In [ ]:
Sigma_nu

In [ ]:
machens_callback_plot(cur_pvec)

In [ ]:
with open('machens_20171206T072804.pkl', 'r') as f:
    [y, all_results, init_paramvec, dict_ind, dict_shape, opt_params, bnds, log_transformed] = pickle.load(f)

In [ ]:
cur_pvec = all_results[-1].x

In [ ]:
machens_callback_plot(init_paramvec[opt_params])

In [ ]:
for i in range(len(all_results)):
    machens_callback_plot(all_results[i].x)

In [ ]:
cur_pvec

In [ ]:
len(all_results)

# Evaluation metrics

In [ ]:
with open('Experiment_machens_results/machens_decision_20181205T144909.pkl', 'rb') as f:
    [y_train, x_train,
     cur_result, simParams,
     init_paramvec, dict_ind, dict_shape, opt_params, 
     bnds, transforms] = pickle.load(f)

In [ ]:
import copy
# Load held out data
maxTrials = len(curData['data']['rt'])
testSimParams = copy.deepcopy(simParams)
testSimParams['Ntrain'] = maxTrials

y = createData(curData, testSimParams, batchnum=0)

y_test = y[:,:,simParams['Ntrain']:]
y = y[:,:,:simParams['Ntrain']]

In [ ]:
# # Load held out data
# np.random.seed(simParams['seed'])
# x_test = tmp['y_python'][:,0::simParams['delta_t'],simParams['Ntrain']:]*simParams['rescale'] # Rescaling to reduce numerical instability
# y_test = x_test + np.sqrt(simParams['Sigma_y'])*np.random.randn(*x_test.shape)


### One step ahead given data

In [ ]:
# Perform linear AR(1) on y as excitation
Dy = y_test.shape[0]
y_t = y[:,1:,:]
y_t1 = y[:,:-1,:]

y_t_rsh = np.reshape(y_t, (y.shape[0], -1)).T
y_t1_rsh = np.reshape(y_t1, (y.shape[0], -1)).T

# Get least squares linear regression weights from original dataset
weights_AR = np.dot(np.dot(np.linalg.inv(np.dot(y_t1_rsh.T, y_t1_rsh)), y_t1_rsh.T), y_t_rsh)

# Get optimal linear estimate of y_tt given the y_t-1 vector
y_test_t1_rsh = np.reshape(y_test[:,:-1,:], (y.shape[0], -1)).T
y_test_t_hat = np.dot(y_test_t1_rsh, weights_AR)

y_test_t_hat = np.reshape(y_test_t_hat.T, (y_test.shape[0], y_test.shape[1]-1, y_test.shape[2]))
y_test_t_hat = np.concatenate([np.zeros((y_test.shape[0],1,y_test.shape[2])), y_test_t_hat], axis=1)
y_test_t_hat.shape

In [ ]:
Sigma_eps

In [ ]:
weights_AR

In [ ]:
# Run the smoothing on this data, then check for RMSE against true x
# res = [obj, x_t1, x_t, sig_t1, sig_t, negll_all]
res_heldout_init = time_full_iter(init_paramvec, 
                                               y_test, dict_ind, dict_shape,  transforms=transforms, ret_smoothed=True)
res_heldout_final = time_full_iter(replace_params(cur_result[-1].x, opt_params, init_paramvec), 
                                               y_test, dict_ind, dict_shape, transforms=transforms, ret_smoothed=True)


In [ ]:
res_heldout_init[0]

In [ ]:
res_heldout_final[0]

In [ ]:
# Get error values between x_t1 and true x
RMSE_AR = np.sqrt(np.mean(np.mean((y_test_t_hat-x_test)**2, axis=2), axis=0))
RMSE_final = np.sqrt(np.mean(np.mean((res_heldout_final[1]-x_test)**2, axis=2), axis=0))
RMSE_init = np.sqrt(np.mean(np.mean((res_heldout_init[1]-x_test)**2, axis=2), axis=0))

plt([
        plt_type.Scatter(y=np.squeeze(RMSE_final), name='RMSE_final'),
        plt_type.Scatter(y=np.squeeze(RMSE_init), name='RMSE_init'),
        plt_type.Scatter(y=np.squeeze(RMSE_AR), name='RMSE_AR')
    ])

### Run forward given initial data point on each trial

In [ ]:
# Get estimate of true variance (around fixed point)
x_var_true = np.var(np.reshape(x[:,x.shape[1]-10:x.shape[1],:] - x[:,x.shape[1]-11:x.shape[1]-1,:], (x.shape[0], -1)).T, axis = 0)
x_var_true = x_var_true[:,None]
x_var_true

In [ ]:
# Copied from time_full_iter

Dy = y.shape[0]
T = y.shape[1]
Ny = y.shape[2]
    
paramvec = replace_params(all_results[-1].x, opt_params, init_paramvec)
paramdict_ind = dict_ind
paramdict_shape = dict_shape

# Transform the parameters back from log-space
if log_transformed is not None:
    inds = (np.arange(paramvec.shape[0]))
    inds = np.setdiff1d(inds,log_transformed)
    out = np.concatenate([np.exp(paramvec[log_transformed]), paramvec[inds]])
    out = out[np.argsort(np.concatenate([log_transformed, inds]))]
    paramvec = out


# Unpack the usual parameters
param_tuple = vec_to_params(paramvec, paramdict_ind, paramdict_shape)
(Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J)  = \
    tuple(list(param_tuple)[:12])

if np.any(np.isnan(lengthscales)):
    set_trace()

# Deal with the extra possible parameters
Sigma_s = None; Sigma_J=None;
if 'Sigma_s' in paramdict_ind.keys():
    Sigma_s = np.reshape(paramvec[paramdict_ind['Sigma_s']], paramdict_shape['Sigma_s'])
if 'Sigma_J' in paramdict_ind.keys():
    Sigma_J = np.reshape(paramvec[paramdict_ind['Sigma_J']], paramdict_shape['Sigma_J'])

L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=lengthscales, z=z, u=u, s=s, J=J, 
                                         sig_eps=Sigma_eps, sig_u = Sigma_u, sig_s=Sigma_s, sig_J=Sigma_J)

# Collect smoothed latent trajectories
x_all_t1 = np.zeros(x_test.shape)
sig_all_t1 = np.zeros((x_test.shape[0]**2, x_test.shape[1], x_test.shape[2]))

for n in range(x_test.shape[2]):
    mu_t1_t1 = mu_0_0
    Sigma_t1_t1 = Sigma_0_0
        
    for t in range(x_test.shape[1]):
        mu_t_t1, Sigma_t_t1 = update_t_t1(mu_t1_t1, Sigma_t1_t1, L, targets, kernel_variance, 
                                          Sigma_eps, z, u, lengthscales, s, J)
        
        x_all_t1[:,t,n] = mu_t_t1.flatten()            
        sig_all_t1[:,t,n] = np.reshape(Sigma_t_t1,(1,-1))
        
        cur_nll_term, mu_t1_t1, Sigma_t1_t1 = update_t_t(mu_t_t1, Sigma_t_t1, C, Sigma_nu, y_test[:,t:(t+1),n])
        #print [mu_t1_t1, y_test[:,t:(t+1),n]]
        Sigma_t1_t1 = np.diag(Sigma_t1_t1)[:,None]
        if t>2:
            # Integrate the first Tau data points, but then no more data available (ignore mu_t1_t1, do we add noise?)
            mu_t1_t1 = mu_t_t1 # + np.sqrt(Sigma_eps)*np.random.randn(*mu_t_t1.shape)

        # Check "condition number" and add diag term to correct if needed            
        if (np.min(Sigma_t1_t1)<1e-6):
            Sigma_t1_t1 = Sigma_nu

In [ ]:
plot_trajectories(y_test,x_test,
                      x_all_t1, x_all_t1, sig_all_t1, sig_all_t1,
                      1, marker_op=0.3)

In [ ]:
machens_callback_plot(all_results[-1].x)